In [1]:
import math
import random
import time
import numpy as np
import scipy as sc
from sklearn import preprocessing

In [66]:
delta = 0.001
epsilon = 10**-6

In [2]:
def direct_evaluation(S,T,weights,delta):
    _,d = weights.shape
    checkPot = []
    for target in T:
        # for each source, add to potential at current target
        s_index = np.any(S != target,axis=1)
        S = S[s_index,:]
        weights = weights[s_index,:]
        norm = np.linalg.norm(S-target, axis=1)
        potential = np.exp(-norm**2 /delta)[:, np.newaxis]*weights
        cum_potential = np.sum(potential,axis=0)
        checkPot.append(cum_potential)

    checkPot = np.sort(checkPot)
    return checkPot


In [67]:
#Direct Evaluation Test with sphere data
X = np.loadtxt('data/sphere_x.txt')
Y = np.loadtxt('data/sphere_y.txt')
Z = np.loadtxt('data/sphere_z.txt')

X0 = np.dstack((X,Y,Z))[0]

X = np.loadtxt('data/alpha0_x.txt')
Y = np.loadtxt('data/alpha0_y.txt')
Z = np.loadtxt('data/alpha0_z.txt')

alpha0 = np.dstack((X,Y,Z))[0]

#Calculating direct Evaluation
%timeit cp = direct_evaluation(X0, X0, alpha0, delta)

1 loops, best of 3: 1.89 s per loop


In [4]:
#Reference: http://stackoverflow.com/a/1235363
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.

    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.

    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.

    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])

    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [52]:
class targetBox:

    def __init__(self, index, Nside, T, T_index, d, p):
        self.index = index
        self.left = index/Nside
        self.right = (index+1)/Nside
        self.center = (index+0.5)/Nside
        self.targets,self.targets_values = self.getTargets(T,T_index)
        self.len = len(self.targets)
        self.potentials = np.zeros((len(self.targets),d)) # potential at  target
        self.Taylor = np.zeros((p,p,d)) # Taylor coefficients for box

    # identical to getSources
    def getTargets(self, T,T_index):
        left = np.all( self.left<= T_index ,axis = 1)
        right = np.all(T_index < self.right, axis= 1)
        targets_index = np.logical_and(left,right)
        targets_values = T[targets_index]
        targets = T_index[targets_index]

        return targets, targets_values 


    def printTargets(self):
        print 'Targets in Box', self.index
        print self.targets


In [53]:
class sourceBox:

    def __init__(self, index, Nside, S, S_index, weights, p):
        self.index = index # box index
        self.left = index/Nside # left endpoints in each dimension of box
        self.right = (index+1)/Nside # right endpoints
        self.center = (index+0.5)/Nside # center of box
        self.sources, self.sources_values,self.weights = self.getSources(S,S_index,weights)
        self.len = len(self.sources)
        self.Hermite = np.zeros((p,p))# Hermitian coefficients for box

    def getSources(self, S, S_index,weights):
        '''
            getSources iterates through the list of sources, finding sources that belong in the box
            and removing them from the list of sources so that after all boxes the source list
            will be empty
        '''
        left = np.all( self.left<= S_index ,axis = 1)
        right = np.all(S_index < self.right, axis= 1)
        sources_index = np.logical_and(left,right)
        sources_values = S[sources_index]
        sources = S_index[sources_index]
        weights = weights[sources_index,:]

        return sources, sources_values, weights

    def printSources(self):
        print 'Sources in Box', self.index
        print self.sources

In [ ]:

            else:
            # form the Hermite coefficients
                for arr in indexes:
                        sum = 0
                        for sourceInd, source in enumerate(B.sources_values):
                            sum += B.weights[sourceInd]*(np.prod(np.power((source - B.center)/sqrt(delta)),arr))
                        B.Hermite[i][j] = sum/(np.prod(arr))        # NB > NF
        #BORRAR: Si hay muchos sources
        else:
            # form the Hermite coefficients
            for i in range(p):
                for j in range(p):
                    sum = 0
                    for sourceInd, source in enumerate(B.sources):
                        sum += B.weights[sourceInd]*((source[0] - B.center[0])/sqrt(delta))**i*((source[1] - B.center[1])/sqrt(delta))**j

                    B.Hermite[i][j] = sum/(fact(i)*fact(j))

#####################################################################
#            print 'testing Hermite coefficients,', B.index
#            directsum = 0
#            for sourceInd, source in enumerate(B.sources):
#                 directsum += B.weights[sourceInd]*exp(-norm([1,0], source)**2/delta)
#            print 'directsum =', directsum
#
#            h = hermite([1,0], B.center, delta, p, d)
#            hermitesum = 0
#            for i in range(p):
#                for j in range(p):
#                    hermitesum += B.Hermite[i][j]*h[0][i]*h[1][j]
#
#            print 'hermitesum =', hermitesum
#
#####################################################################

            for C in BT:
                # if C is in interaction list of B
                if (abs(B.index[0] - C.index[0]) <= n) and (abs(B.index[1] - C.index[1]) <= n):
                    Total += 1
                    MC = len(C.targets)

                    # evaluate Hermite expansion at each target
                    if MC <= ML:
                        #print 'Computing B', B.index, 'on C', C.index, 'by 3 (Hermite for B, evaluate at target)'
                        by3 += 1

                        for targetInd, target in enumerate(C.targets):
                            # for each target create Hermite functions using recursion
                            h = hermite(target, B.center, delta, p, d)

                            # for each target add to the potential the effect from the box, indexed over terms in expansion
                            for i in range(p):
                                for j in range(p):
                                    C.potentials[targetInd] += B.Hermite[i][j]*h[0][i]*h[1][j]

                    # Hermite expansion in B and Taylor series about C
                    else:
                        #print 'Computing B', B.index, 'on C', C.index, 'by 4 (Hermite for B, Taylor about C)'
                        by4 += 1

                        # create the Hermite functions between B and C
                        h = hermite(C.center, B.center, delta, 2*p, d)
                        
                        # i, j index the Taylor series coefficient we add to
                        for i in range(p):
                            for j in range(p):
                                sum = 0

                                # k, l index the Hermite expansion coefficients we computed
                                for k in range(p):
                                    for l in range(p):
                                        sum += B.Hermite[k][l]*h[0][i+k]*h[1][j+l]

                                C.Taylor[i][j] += (-1)**(i+j)/(fact(i)*fact(j))*sum

    # Sum up Taylor coefficients
    for C in BT:
        if len(C.targets) > ML:
            for targetInd, target in enumerate(C.targets):
                sum = 0

                for i in range(p):
                    for j in range(p):
                        sum += C.Taylor[i][j]*((target[0] - C.center[0])/sqrt(delta))**i*((target[1] - C.center[1])/sqrt(delta))**j

                C.potentials[targetInd] += sum

    # stop timing
    elapsedFG = time.clock() - start

    # gather potentials to check error
    Pot = []
    for C in BT:
        for targetInd, target in enumerate(C.targets):
            Pot.append(target+[C.potentials[targetInd]])

    Pot.sort()

    for index, line in enumerate(checkPot):
        line.append(Pot[index][2])

    #print checkPot
    # compute relative error
    sum = 0
    for line in checkPot:
        sum += abs(line[2]-line[3])/line[2]
    relError = sum/len(checkPot)

    print 'r =', r
    print 'Nside =', Nside
    print 'n =', n
    print 'p =', p
    print 'Relative Error =', relError
    print Total, 'interactions total'
    print 'solved', by1, 'interactions by 1: direct evaluation'
    print 'solved', by2, 'interactions by 2: Taylor expansion'
    print 'solved', by3, 'interactions by 3: Hermite expansion and evaluate'
    print 'solved', by4, 'interactions by 4: Hermite then Taylor'

    print 'direct solve seconds:', elapsedDirect
    print 'FG solve seconds:', elapsedFG
    print '-----------------------------------------------------------------------------------------'

#    print 'Printing Fast Gauss targets and potentials'
#    for line in Pot:
#        print line
    
    return [elapsedDirect, elapsedFG, relError]



In [7]:
def fgt_direct_eval(S,T,weights,delta):
    _,d = weights.shape
    cum_pot = np.zeros((len(T),d))
    for i,target in enumerate(T):
        s_index = np.any(S != target,axis=1)
        S = S[s_index,:]
        weights = weights[s_index,:]

        # for each source, add to potential at current target
        norm = np.linalg.norm(S-target, axis=1)

        potential = np.exp(-norm**2 /delta)[:, np.newaxis]*weights
        cum_potential = np.sum(potential,axis=0)
        cum_pot[i]=cum_potential
    return cum_pot

In [47]:
def hermite(x, y, delta, p, d):
    # computes Hermite functions using recursion
    h = np.empty((p, y.shape[0], d))

    diff = (x - y) / np.sqrt(delta)
    h[0,:,:] =  np.exp(-(diff)**2)
    h[1,:,:] = 2 * diff * h[0]
    
    for l in range(2,p):
        h[l] = 2* diff * h[l-1] - 2 * (l-1) * h[l-2]

    #for k in range(d):
    #    h[k][0] = np.exp(-((x[k] - y[k])/np.sqrt(delta))**2)
    #    h[k][1] = 2*(x[k] - y[k])/np.sqrt(delta)*h[k][0]
        
    #    for l in range(2,p):
    #        h[k][l] = 2*(x[k] - y[k])/np.sqrt(delta)*h[k][l-1] - 2*(l-1)*h[k][l-2]
    return h

In [68]:
def FastGauss(X, weights, delta, epsilon):
    """
        Implements the Fast Gauss Transform.

        Inputs:
            S - Sources
            T - Targets
            delta - variance
            epsilon - tolerance (NOTE: results only yield about half the digits that tolerance specifies)
    """
    S = X.copy()
    T = X

    _,d = X.shape
    
    r = 0.5
    Nside = np.ceil(1/(r*np.sqrt(2*delta)))
    r = 1/(Nside*np.sqrt(2*delta)) 
    n = int(np.ceil(np.sqrt(-np.log(epsilon)/(2*r**2)))) 
    K = 1.09**d
    
    p = 1
    while (K*(1/sc.special.factorial(p))**(d/2)*(r**(p+1)/(1-r))**d > epsilon): #BORRAR: Calculando Precision
        p += 1
 

    # cutoff values, use direct evaluation or plug in for boxes with sources (NF) or targets (ML) below number
    NF = p**d
    ML = p**d

    #Transforming data to [0,1]
    minmax = preprocessing.MinMaxScaler()
    minmax.fit(X)

    X_index = minmax.transform(X)
    S_index = X_index.copy()
    T_index = X_index
    # start timing Fast Gauss
    start = time.clock()

    indexes = cartesian(np.tile(np.arange(Nside, dtype= np.dtype(np.int16)),(d,1)))
    
    BS = [sourceBox(arr, Nside, S, S_index, weights, p) for arr in indexes]  
    BT = [targetBox(arr, Nside, T, T_index, d, p)  for arr in indexes]  
    
    for B in BS:  
        NB = B.len
        # use Gaussian for each source in B
        if NB <= NF:
            for C in BT:
                if (np.all(np.abs(B.index - C.index) <= n)):
                    MC = C.len 
                    # directly compute each source's effect on potential of each target
                    if MC <= ML:
                        C.potentials += fgt_direct_eval(B.sources_values,C.targets_values,B.weights,delta)
                    else:
                        #Change to vectorial
                        for arr in indexes:           
                            # for each source create Hermite functions using recursion
                            h = hermite(minmax.inverse_transform(C.center), B.sources_values, delta, p, d)
                            #C.Taylor[arr] += B.weights*(-1)**(np.sum(arr))/(np.prod(sc.special.factorial(arr)))*h[0][arr[0]]*h[1][arr[1]]                                    

In [69]:
%timeit FastGauss(X0, alpha0, delta, epsilon)

KeyboardInterrupt: 